In [56]:
import math
import random
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

In [57]:
df = pd.read_csv("../wavData.csv")
#drop columns unimportant to modeling
df.drop(['filename', 'frameID'], axis=1, inplace=True)
df['intelligence'] = df['intelligence'].map({"Human": 0, "AI": 1})
df.drop(['intelligence'], axis=1, inplace=False)
csv_data = df.values.tolist()

In [58]:
#split up training and test data 
#8planned 80/20 split
def splitData(data, ratio= 0.8):
    train_num = int(len(data) * ratio)
    training = []
    testing = list(data)
    while len(training) < train_num:
        index = random.randrange(len(testing))
        training.append(testing.pop(index))
    return training, testing

#Groups the data by class, key = class value = list of data for that class 
def groupByClass(data):
    ddict = {}
    for i in range(len(data)):
        if data[i][-1] not in ddict:
            ddict[data[i][-1]] = []
        ddict[data[i][-1]]
    return ddict

In [59]:
#calculate mean 
def Mean(num):
    m = np.mean(num)
    return m

#calculate standard deviation
def stdDev(num):
    stdev = np.std(num)
    return stdev

#calculate mean and standard deviation for a given class 
def classMeanAndStdDev(data):
    info = {}
    ddict = groupByClass(data)
    for cv, istnce in ddict.items():
        info[cv] = [classMeanAndStdDev(attribute) for attribute in zip(*istnce)]
    return info

In [60]:
#Calulate the Probability
def Probability(x, mean, stdev):
    expo = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(2 * stdev, 2))))
    retval = (1 / (math.sqrt(2 * math.pi) * (stdev))) * expo
    return retval

#claculates the Gaussian probabilty given a class 
def ClassGProbability(infoDict, testData):
    pb = {} # dictonary to store the probabilites 
    for cv, cs in infoDict.items():
        pb[cv] = 1
        for i in range(len(cs)):
            mean, std_dev = cs[i]
            x = testData[i]
            pb[cv] *= Probability(x, mean, std_dev)
    return pb

In [ ]:
#get the class with the higest probability 
def ClassWithHighestProb(infoDict, test):
    probability = ClassGProbability(infoDict,test)
    hprob = max(probability, probability.get)
    return hprob

In [62]:
#Classify the data then make values into deciamls
def ClassifyData():
    dta = groupByClass(csv_data)
    for i in range(len(dta)):
        for j in range(len(dta[i]) - 1):
            dta[i][j] = float(dta[i][j])

In [ ]:
training, test = splitData(csv_data,ratio=0.8)
print("Amount of training data",len(training))

Training data 4482
[[0.0, 0.92975444, 0.7576904, 0.2467948717948718, 0.9123129, 0.41923195, 0.05075424, 0.59768814, 0.34250823, 0.22953424, 0.5555922, 0.57649565, 0.28457296, 0.485928, 0.7268603, 0.35771048, 0.25247675, 0.38415533, 0.45605075, 0.48845816, 0.39091912, 0.7004058, 0.53007, 0.43463087, 0.2160353985796626, 0.2263742502886118, 0.2966101694915254], [1.0, 0.031904347, 0.03027317, 0.9932773109243698, 0.524713, 0.12984493, 0.80094457, 0.5927898, 0.82541144, 0.61935794, 0.78643227, 0.74494904, 0.5297132, 0.66451937, 0.47116023, 0.7356529, 0.6124351, 0.5189257, 0.7233081, 0.50694174, 0.58745575, 0.4206996, 0.67404926, 0.7253181, 0.9389971405989592, 0.7938159066609705, 0.9658536585365852], [0.0, 0.030494979, 0.031181967, 0.0618421052631578, 0.38473955, 0.6176091, 0.503739, 0.57258886, 0.56311715, 0.5438229, 0.54007894, 0.4468425, 0.6116181, 0.5410197, 0.8361093, 0.6178435, 0.36555108, 0.34309754, 0.25582984, 0.65175253, 0.7852369, 0.36811203, 0.3471446, 0.41339272, 0.15215753731590

In [ ]:
print("Amount of test data",len(test))

test data 1121
[0.0, 0.029111018, 0.029648047, 0.2745664739884393, 0.07598244, 0.589434, 0.37764737, 0.5763174, 0.25313962, 0.69690126, 0.6840709, 0.8474432, 0.78108764, 0.28892773, 0.65510124, 0.65503454, 0.7585805, 0.5331738, 0.65082145, 0.26431715, 0.34279186, 0.27836078, 0.79970104, 0.4830606, 0.2910400569103776, 0.279648703297275, 0.2891566265060241] [0.0, 0.029111018, 0.042420767, 0.2312138728323699, 0.07953742, 0.6955891, 0.5226309, 0.73403597, 0.29383096, 0.698121, 0.68549174, 0.8527216, 0.74313545, 0.22731668, 0.4938836, 0.50844187, 0.6191948, 0.30933443, 0.5507545, 0.14441438, 0.23256257, 0.15527572, 0.8209354, 0.5741529, 0.2075650223828297, 0.2780139510709738, 0.2620481927710843] [0.0, 0.044762105, 0.06382481, 0.1965317919075144, 0.118932806, 0.766647, 0.5534389, 0.78450227, 0.31816489, 0.7049367, 0.68479025, 0.85795695, 0.7020569, 0.28481537, 0.4811943, 0.4511385, 0.54567546, 0.29551217, 0.54145604, 0.038545396, 0.2896629, 0.18423823, 0.7859874, 0.5505307, 0.166419348968545

In [ ]:
tmod = classMeanAndStdDev(training)